In [25]:
#r "nuget:Microsoft.ML,1.5.0-preview2"
#r "nuget:Microsoft.ML.TimeSeries,1.5.0-preview2"
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.TimeSeries;

Installing package Microsoft.ML, version 1.5.0-preview2.....

Installing package NodaTime, version 2.4.6.....

Installing package Octokit, version 0.32.0.....

Installing package Microsoft.ML.TimeSeries, version 1.5.0-preview2.....

In [86]:
private const string DATA_FILEPATH = "./covid_19_Hubei.csv";
private static readonly MLContext mlContext = new MLContext();

In [87]:
public class ModelInput
{
    [LoadColumn(0)]
    public float Confirmed { get; set; }

    [LoadColumn(1)]
    public float Deaths { get; set; }

    [LoadColumn(2)]
    public float Recovered { get; set; }
}

public class ModelOutput
{
    public float[] Forecasted { get; set; }
}

In [88]:
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATA_FILEPATH,
    hasHeader: true,
    separatorChar: ',',
    allowQuoting: true);

In [89]:
var dataToView = data.Preview().RowView.Select(r => (float)r.Values[0].Value).ToList();

var infectionHistogram = Chart.Plot(new Graph.Bar(){y = dataToView});
var layout = new Layout.Layout(){title="Distribution of infections per day"};
infectionHistogram.WithLayout(layout);
display(infectionHistogram);

In [96]:
var numSeriesDataPoints = 50;

IEstimator<ITransformer> forecastEstimator = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: nameof(ModelOutput.Forecasted),
    inputColumnName: nameof(ModelInput.Confirmed),
    windowSize: 15,
    seriesLength: 25,
    trainSize: numSeriesDataPoints,
    horizon: 14);

ITransformer forecastTransformer = forecastEstimator.Fit(data);

var forecastEngine = forecastTransformer.CreateTimeSeriesEngine<ModelInput, ModelOutput>(mlContext);
var prediction = forecastEngine.Predict();

In [97]:
int[] elements = Enumerable.Range(0, numSeriesDataPoints).ToArray();
int[] predictedElements = Enumerable.Range(numSeriesDataPoints, numSeriesDataPoints + 10).ToArray();

In [98]:
var predictedIinfectionHistogram = Chart.Plot(new[] { new Graph.Bar(){x = elements, y = dataToView}, new Graph.Bar(){x = predictedElements, y = prediction.Forecasted} });
predictedIinfectionHistogram.WithLayout(layout);
display(predictedIinfectionHistogram);
